In [ ]:
# ==========================================
# 2. 데이터셋 및 Collate 함수 (DPO용)
# DPO는 (프롬프트, 선택된 답변, 거부된 답변)의 쌍이 필요합니다.
# ==========================================
class PreferenceDataset(Dataset):
    def __init__(self, data, tokenizer):
        
        self.data = data
        self.encoded_texts = []
        
        # 데이터를 미리 토큰화(Tokenization)하여 저장합니다.
        for entry in data:
            prompt = format_input(entry)
            rejected_response = entry["rejected"] # 👎 덜 선호되는 답변
            chosen_response = entry["chosen"]     # 👍 더 선호되는 답변

            # 프롬프트 부분만 따로 인코딩 (나중에 마스킹하기 위해 필요)
            prompt_tokens = tokenizer.encode(prompt)
            
            # 프롬프트 + 답변 형태로 전체 문장을 인코딩
            chosen_full_tokens = tokenizer.encode(f"{prompt}\n\n### Response:\n{chosen_response}")
            rejected_full_tokens = tokenizer.encode(f"{prompt}\n\n### Response:\n{rejected_response}")

            self.encoded_texts.append({
                "prompt": prompt_tokens,
                "chosen": chosen_full_tokens,
                "rejected": rejected_full_tokens,
            })

In [ ]:
def custom_collate_fn(batch, pad_token_id=50256, allowed_max_length=None, mask_prompt_tokens=True, device="cpu"):
    """
    배치(Batch) 내의 데이터 길이를 맞추고(Padding), 마스크(Mask)를 생성하는 함수.
    """
    batch_data = {
        "prompt": [],
        "chosen": [],
        "rejected": [],
        "rejected_mask": [], # Loss 계산 시 무시할 부분 (패딩 등)
        "chosen_mask": []
    }
    
    # 1. 배치 내에서 가장 긴 시퀀스 길이 찾기 (패딩을 위해)
    max_length_common = 0
    if batch:
        for key in ["chosen", "rejected"]:
            current_max = max(len(item[key]) + 1 for item in batch)
            max_length_common = max(max_length_common, current_max)
            
            # 2. 패딩 추가 및 마스크 생성
    for item in batch:
        prompt = torch.tensor(item["prompt"])
        batch_data["prompt"].append(prompt)

        for key in ["chosen", "rejected"]:
            sequence = item[key]
            # 최대 길이에 맞춰 패딩 토큰 추가
            padded = sequence + [pad_token_id] * (max_length_common - len(sequence))
            
            # 기본 마스크: 데이터가 있는 곳은 1(True), 패딩은 0(False)
            mask = torch.ones(len(padded)).bool()
            mask[len(sequence):] = False  # 패딩 부분 마스킹

            # [중요] 프롬프트 부분 마스킹 (선택적)
            # DPO는 '답변'의 확률 차이를 학습하므로, 질문(Prompt) 부분은 Loss 계산에서 제외합니다.
            if mask_prompt_tokens:
                mask[:prompt.shape[0] + 2] = False 

            batch_data[key].append(torch.tensor(padded))
            batch_data[f"{key}_mask"].append(mask)
            
    # 3. 텐서 변환 및 디바이스(GPU/CPU) 이동
    for key in ["chosen", "rejected", "chosen_mask", "rejected_mask"]:
        tensor_stack = torch.stack(batch_data[key])
        # 길이 제한이 있다면 자르기
        if allowed_max_length is not None:
            tensor_stack = tensor_stack[:, :allowed_max_length]
        batch_data[key] = tensor_stack.to(device)

    return batch_data

In [ ]:
# ==========================================
# 3. DPO Loss 및 Log Probability 계산 함수
# DPO의 핵심 수학적 로직이 들어있는 부분.
# ==========================================
def compute_logprobs(logits, labels, selection_mask=None):
    """
    모델의 출력(logits)과 정답(labels)을 받아 해당 정답 토큰의 로그 확률을 계산.
    """
    # Auto-regressive 모델 특성상, 입력 [A, B, C]에 대해 예측은 [B, C, D]가 되므로 시프트(Shift)
    labels = labels[:, 1:].clone()
    logits = logits[:, :-1, :]
    
    log_probs = F.log_softmax(logits, dim=-1)
    
    # 실제 정답 레이블에 해당하는 확률값만 추출 (gather 사용)
    selected_log_probs = torch.gather(
        input=log_probs,
        dim=-1,
        index=labels.unsqueeze(-1)
    ).squeeze(-1)
    
    if selection_mask is not None:
        # 마스크도 시프트하여 적용 (패딩이나 프롬프트 영역 무시)
        mask = selection_mask[:, 1:].clone()
        selected_log_probs = selected_log_probs * mask
        
        # 유효한 토큰들의 로그 확률 평균 계산
        avg_log_prob = selected_log_probs.sum(-1) / mask.sum(-1)
        return avg_log_prob
    else:
        return selected_log_probs.mean(-1)

In [ ]:
def compute_dpo_loss(model_chosen_logprobs, model_rejected_logprobs, 
                     reference_chosen_logprobs, reference_rejected_logprobs, beta=0.1):
    """
    DPO 손실 함수 계산:
    Policy 모델이 Reference 모델보다 'chosen' 답변을 더 선호하고, 'rejected' 답변을 덜 선호하도록 유도합니다.
    
    beta: Reference 모델에서 얼마나 벗어날지 제어하는 하이퍼파라미터 (보통 0.1~0.5)
    """
    # 모델의 (Chosen - Rejected) 로그 확률 차이
    model_logratios = model_chosen_logprobs - model_rejected_logprobs
    # 기준 모델의 (Chosen - Rejected) 로그 확률 차이
    reference_logratios = reference_chosen_logprobs - reference_rejected_logprobs
    
    # 두 비율의 차이 (Policy가 Reference보다 얼마나 더 잘 구분했는가)
    logits = model_logratios - reference_logratios
    
    # Sigmoid 후 음수 로그 (Cross Entropy와 유사) -> 이 값을 최소화하면 선호도 차이가 극대화됨
    losses = -F.logsigmoid(beta * logits)
    
    # 학습 추적용 보상(Reward) 계산 (실제 학습엔 안 쓰이고 로깅용)
    chosen_rewards = (model_chosen_logprobs - reference_chosen_logprobs).detach()
    rejected_rewards = (model_rejected_logprobs - reference_rejected_logprobs).detach()
    
    return losses.mean(), chosen_rewards.mean(), rejected_rewards.mean()

In [ ]:
def compute_dpo_loss_batch(batch, policy_model, reference_model, beta):
    """
    하나의 배치에 대해 전체 DPO 과정을 수행하는 헬퍼 함수
    """
    # 1. 학습 중인 모델(Policy Model)의 로그 확률 계산 (기울기 계산 O)
    policy_chosen_log_probas = compute_logprobs(
        logits=policy_model(batch["chosen"]),
        labels=batch["chosen"],
        selection_mask=batch["chosen_mask"]
    )
    policy_rejected_log_probas = compute_logprobs(
        logits=policy_model(batch["rejected"]),
        labels=batch["rejected"],
        selection_mask=batch["rejected_mask"]
    )
    
    # 2. 기준 모델(Reference Model)의 로그 확률 계산 (기울기 계산 X -> 메모리 절약)
    with torch.no_grad():
        ref_chosen_log_probas = compute_logprobs(
            logits=reference_model(batch["chosen"]),
            labels=batch["chosen"],
            selection_mask=batch["chosen_mask"]
        )
        ref_rejected_log_probas = compute_logprobs(
            logits=reference_model(batch["rejected"]),
            labels=batch["rejected"],
            selection_mask=batch["rejected_mask"]
        )

    # 3. 최종 Loss 계산
    loss, chosen_rewards, rejected_rewards = compute_dpo_loss(
        model_chosen_logprobs=policy_chosen_log_probas,
        model_rejected_logprobs=policy_rejected_log_probas,
        reference_chosen_logprobs=ref_chosen_log_probas,
        reference_rejected_logprobs=ref_rejected_log_probas,
        beta=beta
    )
    return loss, chosen_rewards, rejected_rewards


In [ ]:
# ==========================================
# 4. 평가 및 학습 루프
# 실제 모델 학습을 돌리는 메인 루프입니다.
# ==========================================
def evaluate_dpo_loss_loader(policy_model, reference_model, train_loader, val_loader, beta, eval_iter):
    """
    학습 중간에 모델 성능을 평가(Validation)하는 함수
    """
    policy_model.eval() # 평가 모드 전환 (Dropout 등 비활성화)
    with torch.no_grad():
        # 로더를 순회하며 평균 Loss 계산하는 내부 함수
        def compute_loader_metric(loader):
            total_loss, total_chosen, total_rejected = 0., 0., 0.
            num_batches = min(eval_iter, len(loader))
            if num_batches == 0: return float("nan"), float("nan"), float("nan")
            
            for i, batch in enumerate(loader):
                if i >= num_batches: break
                loss, chosen, rejected = compute_dpo_loss_batch(batch, policy_model, reference_model, beta)
                total_loss += loss.item()
                total_chosen += chosen.item()
                total_rejected += rejected.item()
            return total_loss/num_batches, total_chosen/num_batches, total_rejected/num_batches

        train_loss, train_chosen, train_rejected = compute_loader_metric(train_loader)
        val_loss, val_chosen, val_rejected = compute_loader_metric(val_loader)

    policy_model.train() # 다시 학습 모드로 전환
    return {
        "train_loss": train_loss,
        "train_chosen_reward": train_chosen,
        "train_rejected_reward": train_rejected,
        "val_loss": val_loss,
        "val_chosen_reward": val_chosen,
        "val_rejected_reward": val_rejected
    }

In [ ]:
def train_model_dpo_simple(policy_model, reference_model, train_loader, val_loader, 
                           optimizer, num_epochs, beta, eval_freq, eval_iter, start_context, tokenizer):
    """
    전체 학습 과정을 관리하는 함수
    """
    tracking = {"train_losses": [], "val_losses": [], "tokens_seen": []}
    tokens_seen, global_step = 0, -1

    for epoch in range(num_epochs):
        policy_model.train() # 학습 모드 시작
        for batch in train_loader:
            optimizer.zero_grad() # 이전 기울기 초기화
            
            # Loss 계산
            loss, chosen_rewards, rejected_rewards = compute_dpo_loss_batch(
                batch=batch, policy_model=policy_model, reference_model=reference_model, beta=beta
            )
            
            loss.backward() # 역전파 (기울기 계산)
            optimizer.step() # 가중치 업데이트
            
            tokens_seen += batch["chosen"].numel()
            global_step += 1
          

        # 에포크가 끝날 때마다 샘플 생성하여 육안으로 확인
        generate_and_print_sample(model=policy_model, tokenizer=tokenizer, device=loss.device, start_context=start_context)

    return tracking